# This notebook requires the output of  uses the output of Section_6.1_education_empirical_bayes_step.ipynb

In [1]:
import numpy as np
import pandas as pd
from scipy import optimize
import sys
sys.path.append("../src/")
import affine_operator_win_bounds as affine_ops

In [2]:
min_students = 5
fn = "../results/education/means_and_SEs_by_school_min_students=%02d.csv"%min_students
df = pd.read_csv(fn)
y, sigma, mu, tau = df['mean'].values, df.sigma_n.values, df.prior_mean.values, df.prior_sd.values
N = y.shape[0]
Sigma = np.diag(sigma**2)
A, k = np.eye(N), np.zeros(N)
C = np.diag(tau**2/(tau**2 + sigma**2))
l = (sigma**2/(tau**2 + sigma**2))*mu

affine_ops.c_value(A=A, b=k, C=C, d=l,Sigma=Sigma, y=y)

0.9925817329571586

In [3]:
# Do the same thing but permuting the order of hte features.
perm = np.random.permutation(y.shape[0])
affine_ops.c_value(A=A, b=k, C=C, d=l, Sigma=Sigma[perm][:, perm], y=y[perm])

0.0

In [4]:
# compute c-values with different minimum numbers of students
min_students_vals = [2, 10]
for min_students in min_students_vals:
    fn = "../results/education/means_and_SEs_by_school_min_students=%02d.csv"%min_students
    df = pd.read_csv(fn)
    y, sigma, mu, tau = df['mean'].values, df.sigma_n.values, df.prior_mean.values, df.prior_sd.values
    N = y.shape[0]
    Sigma = np.diag(sigma**2)
    A, k = np.eye(N), np.zeros(N)
    C = np.diag(tau**2/(tau**2 + sigma**2))
    l = (sigma**2/(tau**2 + sigma**2))*mu

    c_val = affine_ops.c_value(A=A, b=k, C=C, d=l, Sigma=Sigma, y=y)
    print("min_students = ", min_students, "\tc_value=", c_val)

min_students =  2 	c_value= 0.8769577159362123
min_students =  10 	c_value= 0.9999928992392249


In [5]:
def compute_bayes_est(df):
    y, sigma, mu, tau = df['mean'].values, df.sigma_n.values, df.prior_mean.values, df.prior_sd.values
    C = np.diag(tau**2/(tau**2 + sigma**2))
    l = (sigma**2/(tau**2 + sigma**2))*mu
    return C.dot(y) + l

def where_bya_breaks(df):
    y = df['mean']
    theta = df['school_mean']
    theta_hat = y
    theta_star = compute_bayes_est(df)
    win = np.linalg.norm(theta_hat-theta)**2 - np.linalg.norm(theta_star-theta)**2
    
    N = len(y)
    sigma, mu, tau = df.sigma_n.values, df.prior_mean.values, df.prior_sd.values
    Sigma = np.diag(sigma**2)
    A, k = np.eye(N), np.zeros(N)
    C = np.diag(tau**2/(tau**2 + sigma**2))
    l = (sigma**2/(tau**2 + sigma**2))*mu
    
    bya = lambda alpha: affine_ops.b_bound(A=A, C=C, b=k, d=l, alpha=alpha, y=y, Sigma=Sigma)[0]
    c_lower_bound =  1e-10
    if win>bya(c_lower_bound): return c_lower_bound
    if win<bya(1.-c_lower_bound): return 1.-c_lower_bound
    
    where_breaks = optimize.bisect(
        f=lambda alpha: bya(alpha)-win,
        a=c_lower_bound, b=1.-c_lower_bound, xtol=1e-3)
    return where_breaks

In [6]:
# started at 12:57pm -- expected finish by 2am next day... 
from os import path 
for min_students in [2, 5, 10]:
    by_breaks_fn = "../results/education/by_break_values_min_students=%02d.npy"%min_students
    if not path.exists(by_breaks_fn):
        print("fn=",by_breaks_fn)
        by_breaks = []
        for rep in range(1, 5001):
            fn = "../results/education/simulation_min_students=%02d_%03d.csv"%(min_students, rep)
            df = pd.read_csv(fn)

            by_break = where_bya_breaks(df)
            by_breaks.append(by_break)
            if rep%25 ==0: print("rep %03d/%03d"%(rep, 250))

        by_breaks = np.array(by_breaks)
        np.save(by_breaks_fn, by_breaks)